In [9]:
import numpy as np
import gym
from gym.utils import play

def policy_iteration(policy, env, discount_factor=1.0, theta=1e-9, max_iterations=1000):
   
    num_states = env.observation_space.n
    num_actions = env.action_space.n

    value_func = np.zeros(num_states)  # Initialize value function

    for i in range(max_iterations):
        # Policy Evaluation
        while True:
            prev_value_func = value_func.copy()
            for state in range(num_states):
                value_func[state] = sum([policy[state][action] *
                                         sum([prob * (reward + discount_factor * prev_value_func[next_state])
                                              for prob, next_state, reward, _ in env.P[state][action]])
                                         for action in range(num_actions)])
            max_diff = np.max(np.abs(value_func - prev_value_func))
            if max_diff < theta:
                break

        # Policy Improvement
        policy_stable = True
        for state in range(num_states):
            old_action = np.argmax(policy[state])
            best_action_value = max([sum([prob * (reward + discount_factor * value_func[next_state])
                                          for prob, next_state, reward, _ in env.P[state][action]])
                                     for action in range(num_actions)])
            best_actions = [action for action in range(num_actions)
                             if sum([prob * (reward + discount_factor * value_func[next_state])
                                     for prob, next_state, reward, _ in env.P[state][action]]) == best_action_value]
            new_policy = np.zeros(num_actions)
            new_policy[best_actions] = 1 / len(best_actions)
            policy[state] = new_policy
            if old_action != np.argmax(policy[state]):
                policy_stable = False

        if policy_stable:
            break

    return policy, value_func

def value_iteration(env, discount_factor=1.0, theta=1e-9, max_iterations=1000):
   
    num_states = env.observation_space.n
    num_actions = env.action_space.n

    value_func = np.zeros(num_states)  # Initialize value function
    policy = np.zeros((num_states, num_actions))  # Initialize policy

    for i in range(max_iterations):
        prev_value_func = value_func.copy()
        for state in range(num_states):
            action_values = [sum([prob * (reward + discount_factor * prev_value_func[next_state])
                                  for prob, next_state, reward, _ in env.P[state][action]])
                             for action in range(num_actions)]
            value_func[state] = max(action_values)

        max_diff = np.max(np.abs(value_func - prev_value_func))
        if max_diff < theta:
            break

    # Extract optimal policy
    for state in range(num_states):
        action_values = [sum([prob * (reward + discount_factor * value_func[next_state])
                              for prob, next_state, reward, _ in env.P[state][action]])
                         for action in range(num_actions)]
        best_actions = np.argwhere(action_values == np.max(action_values)).flatten()
        policy[state][best_actions] = 1 / len(best_actions)

    return policy, value_func

def compare_algorithms(env):
    """
    Compare the performance of Policy Iteration and Value Iteration algorithms.

    Args:
        env (gym.Env): OpenAI Gym environment object
    """
    num_states = env.observation_space.n
    num_actions = env.action_space.n

    # Initialize random policy
    initial_policy = np.ones((num_states, num_actions)) / num_actions

    # Policy Iteration
    policy_pi, value_func_pi = policy_iteration(initial_policy, env)

    # Value Iteration
    policy_vi, value_func_vi = value_iteration(env)

    # Evaluate algorithms
    num_episodes = 1000
    wins_pi = 0
    wins_vi = 0
    returns_pi = []
    returns_vi = []

    for episode in range(num_episodes):
        state = env.reset()
        done = False
        total_return_pi = 0
        total_return_vi = 0

        while not done:
            action_pi = np.random.choice(num_actions, p=policy_pi[state])
            action_vi = np.random.choice(num_actions, p=policy_vi[state])
            next_state_pi, reward_pi, done, _ = env.step(action_pi)
            next_state_vi, reward_vi, done, _ = env.step(action_vi)
            total_return_pi += reward_pi
            total_return_vi += reward_vi
            state = next_state_pi

        if reward_pi == 1:
            wins_pi += 1
        if reward_vi == 1:
            wins_vi += 1
        returns_pi.append(total_return_pi)
        returns_vi.append(total_return_vi)

    avg_return_pi = np.mean(returns_pi)
    avg_return_vi = np.mean(returns_vi)

    print(f"Policy Iteration: Wins = {wins_pi}, Average Return = {avg_return_pi:.3f}")
    print(f"Value Iteration: Wins = {wins_vi}, Average Return = {avg_return_vi:.3f}")
    print("\n")

    if wins_pi > wins_vi:
        print("Policy Iteration performed better in terms of number of wins.")
    elif wins_pi < wins_vi:
        print("Value Iteration performed better in terms of number of wins.")
    else:
        print("Both algorithms performed equally in terms of number of wins.")
        

    if avg_return_pi > avg_return_vi:
        print("Policy Iteration performed better in terms of average return.")
    elif avg_return_pi < avg_return_vi:
        print("Value Iteration performed better in terms of average return.")
    else:
        print("Both algorithms performed equally in terms of average return.")

if __name__ == "__main__":
    env = gym.make("FrozenLake-v1")
    compare_algorithms(env)

Policy Iteration: Wins = 4, Average Return = 0.004
Value Iteration: Wins = 11, Average Return = 0.011


Value Iteration performed better in terms of number of wins.
Value Iteration performed better in terms of average return.


## Why Value Iteration Outperforms Policy Iteration

- **Efficiency in Larger State Spaces:** Value Iteration can handle larger state spaces more efficiently than Policy Iteration, making it more suitable for environments with a large number of states.

- **Handling Stochastic Environments:** The stochastic nature of the FrozenLake-v1 environment, where the agent's actions have a probability of success or failure, contributes to the better performance of Value Iteration.

- **Effective Handling of Stochastic Transitions:** Value Iteration can account for stochastic transitions more effectively than Policy Iteration, which relies on iteratively evaluating and improving the policy.

- **Suitability for Small State Spaces:** Although the FrozenLake-v1 environment has a relatively small state space, Value Iteration's ability to directly compute the optimal value function makes it more suitable for this environment.

# Overview
The Policy Iteration and Value Iteration algorithms are used to find the optimal policy for a Markov Decision Process (MDP) environment.

## Policy Iteration
Policy Iteration is an iterative algorithm used in the field of reinforcement learning to find the optimal policy for a given MDP. It involves two main steps: Policy Evaluation and Policy Improvement.

**Description:** The Policy Iteration algorithm alternates between policy evaluation and policy improvement.

**Process:**
1. **Policy Evaluation:** Compute the value function for the current policy. This step involves iterating over the states of the MDP and updating their values based on the expected rewards and next-state values.
2. **Policy Improvement:** Update the policy based on the value function. This step involves examining each state and selecting actions that maximize the expected cumulative reward, according to the updated value function.

## Value Iteration
Value Iteration is another iterative algorithm used to find the optimal policy for an MDP. Unlike Policy Iteration, Value Iteration directly computes the optimal value function without explicitly maintaining a policy.

**Description:** The Value Iteration algorithm iteratively computes the optimal value function until it converges to the true optimal values.

**Process:** At each iteration, the algorithm updates the value of each state based on the maximum expected cumulative reward achievable from that state, considering all possible actions and their associated rewards and next states. This process continues until the values converge.

**Outcome:** Once the optimal value function is computed, the optimal policy can be derived from it by selecting the action that maximizes the expected cumulative reward from each state.


## <span style="color: #3366cc; font-family: Arial, sans-serif; font-size: 36px; text-shadow: 2px 2px 4px rgba(0,0,0,0.2); animation: fadeInOut 3s infinite;">With Render</span>


In [ ]:
import numpy as np
import gym
from gym.utils import play

def policy_iteration(policy, env, discount_factor=1.0, theta=1e-9, max_iterations=1000):
    """
    Policy Iteration algorithm to find the optimal policy for a given environment.

    Args:
        policy (np.ndarray): 2D array of size (num_states, num_actions) representing the initial policy
        env (gym.Env): OpenAI Gym environment object
        discount_factor (float): MDP discount factor (default: 1.0)
        theta (float): Threshold for value function change (default: 1e-8)
        max_iterations (int): Maximum number of iterations (default: 1000)

    Returns:
        np.ndarray: Optimal policy
        np.ndarray: Value function under the optimal policy
    """
    num_states = env.observation_space.n
    num_actions = env.action_space.n

    value_func = np.zeros(num_states)  # Initialize value function

    for i in range(max_iterations):
        # Policy Evaluation
        while True:
            prev_value_func = value_func.copy()
            for state in range(num_states):
                value_func[state] = sum([policy[state][action] *
                                         sum([prob * (reward + discount_factor * prev_value_func[next_state])
                                              for prob, next_state, reward, _ in env.P[state][action]])
                                         for action in range(num_actions)])
            max_diff = np.max(np.abs(value_func - prev_value_func))
            if max_diff < theta:
                break

        # Policy Improvement
        policy_stable = True
        for state in range(num_states):
            old_action = np.argmax(policy[state])
            best_action_value = max([sum([prob * (reward + discount_factor * value_func[next_state])
                                          for prob, next_state, reward, _ in env.P[state][action]])
                                     for action in range(num_actions)])
            best_actions = [action for action in range(num_actions)
                             if sum([prob * (reward + discount_factor * value_func[next_state])
                                     for prob, next_state, reward, _ in env.P[state][action]]) == best_action_value]
            new_policy = np.zeros(num_actions)
            new_policy[best_actions] = 1 / len(best_actions)
            policy[state] = new_policy
            if old_action != np.argmax(policy[state]):
                policy_stable = False

        if policy_stable:
            break

    return policy, value_func

def value_iteration(env, discount_factor=1.0, theta=1e-9, max_iterations=1000):
    """
    Value Iteration algorithm to find the optimal policy for a given environment.

    Args:
        env (gym.Env): OpenAI Gym environment object
        discount_factor (float): MDP discount factor (default: 1.0)
        theta (float): Threshold for value function change (default: 1e-8)
        max_iterations (int): Maximum number of iterations (default: 1000)

    Returns:
        np.ndarray: Optimal policy
        np.ndarray: Value function under the optimal policy
    """
    num_states = env.observation_space.n
    num_actions = env.action_space.n

    value_func = np.zeros(num_states)  # Initialize value function
    policy = np.zeros((num_states, num_actions))  # Initialize policy

    for i in range(max_iterations):
        prev_value_func = value_func.copy()
        for state in range(num_states):
            action_values = [sum([prob * (reward + discount_factor * prev_value_func[next_state])
                                  for prob, next_state, reward, _ in env.P[state][action]])
                             for action in range(num_actions)]
            value_func[state] = max(action_values)

        max_diff = np.max(np.abs(value_func - prev_value_func))
        if max_diff < theta:
            break

    # Extract optimal policy
    for state in range(num_states):
        action_values = [sum([prob * (reward + discount_factor * value_func[next_state])
                              for prob, next_state, reward, _ in env.P[state][action]])
                         for action in range(num_actions)]
        best_actions = np.argwhere(action_values == np.max(action_values)).flatten()
        policy[state][best_actions] = 1 / len(best_actions)

    return policy, value_func

def compare_algorithms(env):
    """
    Compare the performance of Policy Iteration and Value Iteration algorithms.

    Args:
        env (gym.Env): OpenAI Gym environment object
    """
    num_states = env.observation_space.n
    num_actions = env.action_space.n

    # Initialize random policy
    initial_policy = np.ones((num_states, num_actions)) / num_actions

    # Policy Iteration
    policy_pi, value_func_pi = policy_iteration(initial_policy, env)

    # Value Iteration
    policy_vi, value_func_vi = value_iteration(env)

    # Evaluate algorithms
    num_episodes = 1000
    wins_pi = 0
    wins_vi = 0
    returns_pi = []
    returns_vi = []

    for episode in range(num_episodes):
        state = env.reset()
        done = False
        total_return_pi = 0
        total_return_vi = 0

        while not done:
            env.render()  # Display the environment
            action_pi = np.random.choice(num_actions, p=policy_pi[state])
            action_vi = np.random.choice(num_actions, p=policy_vi[state])
            next_state_pi, reward_pi, done, _ = env.step(action_pi)
            next_state_vi, reward_vi, done, _ = env.step(action_vi)
            total_return_pi += reward_pi
            total_return_vi += reward_vi
            state = next_state_pi

        if reward_pi == 1:
            wins_pi += 1
        if reward_vi == 1:
            wins_vi += 1
        returns_pi.append(total_return_pi)
        returns_vi.append(total_return_vi)

    avg_return_pi = np.mean(returns_pi)
    avg_return_vi = np.mean(returns_vi)

    print(f"Policy Iteration: Wins = {wins_pi}, Average Return = {avg_return_pi:.3f}")
    print(f"Value Iteration: Wins = {wins_vi}, Average Return = {avg_return_vi:.3f}")

    if wins_pi > wins_vi:
        print("Policy Iteration performed better in terms of number of wins because it converges to the optimal policy more efficiently.")
    elif wins_pi < wins_vi:
        print("Value Iteration performed better in terms of number of wins because it can handle larger state spaces more efficiently.")
    else:
        print("Both algorithms performed equally in terms of number of wins.")

    if avg_return_pi > avg_return_vi:
        print("Policy Iteration performed better in terms of average return because it converges to the optimal policy more efficiently.")
    elif avg_return_pi < avg_return_vi:
        print("Value Iteration performed better in terms of average return because it can handle larger state spaces more efficiently.")
    else:
        print("Both algorithms performed equally in terms of average return.")

if __name__ == "__main__":
    env = gym.make("FrozenLake-v1")
    compare_algorithms(env)